In [1]:
import gym
import numpy as np
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, Dense, Conv2D, Flatten
from keras.callbacks import TensorBoard
import random
import math
from time import time
import cv2

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.index = 0
        self.buffer = [None] * max_size
        self.count = 0

    """
    Add state, action, reward and next_state to replay buffer
    Use None for done state
    """
    def add(self, state, action, reward, next_state):
        self.buffer[self.index] = (state, action, reward, next_state)
        self.index = (self.index + 1) % self.max_size
        self.count = min(self.count + 1, self.max_size)

    def __len__(self):
        """
        Return current item count in replay buffer
        """
        return self.count

    def sample(self, count):
        """
        Sample `count` items from replay buffer
        Filter None items
        So result array may contain less items than count
        """
        count = min(count, self.count)
        samples = [s for s in self.buffer if s is not None]
        return random.sample(samples, count)

In [3]:
class StackFrame:
    def __init__(self, stack_size):
        self.stack_size = stack_size
        self._buffer = []

    def is_full(self):
        return len(self._buffer) == self.stack_size

    def add_item(self, item):
        if self.is_full():
            self._buffer.pop(0)
        self._buffer.append(item)

    def get_contents(self):
        a = np.asarray(self._buffer)
        return np.swapaxes(a, 0, 2)

In [4]:
BUFFER_SIZE = 100000
BATCH_SIZE = 32

COPY_WEIGHTS_INTERVAL = 10000

IMAGE_SIZE = 84
STACK_SIZE = 4
    
MIN_EPSILON = 0.1
MAX_EPSILON = 1.0
LAMBDA = 0.001
    
GAMMA = 0.99

REPLAY_START_COUNT = 50000

In [5]:
class Learner:
    def __init__(self, learner_type = 'dqn-v0', ob_dimen = 0, act_dimen = 0, buffer_size = BUFFER_SIZE):
        self.learner_type = learner_type
        self.ob_dimen = ob_dimen
        self.act_dimen = act_dimen
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.act_counter = 0
        
        self._curr_stack = StackFrame(STACK_SIZE)
        self._prev_stack = StackFrame(STACK_SIZE)
        
        self.tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
        self.model = self._build_model()
        self.target_model = self._build_model()
        # set same weights initially
        self._update_target_network()
        # only compile model because we don't train on target network, we just use it to preduct q'
        self.model.compile(optimizer='adam', loss='mse')
        
    def _build_model(self):
        input_ob = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,STACK_SIZE), dtype='float32')
        conv1 = Conv2D(32, (8, 8), strides=4, activation='relu', padding='valid')(input_ob)
        conv2 = Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid')(conv1)
        conv3 = Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid')(conv2)
        flatten = Flatten()(conv3)
        fc1 = Dense(512, activation='relu')(flatten)
        out = Dense(self.act_dimen)(fc1)
        model = Model(inputs=input_ob, outputs=out)
        return model
        
    def _predict(self):
        return self.model.predict(np.asarray([self._curr_stack.get_contents()]))
    
    def act(self, ob):
        self.act_counter += 1
        
        epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.act_counter)
        rand = random.random()
        if len(self.replay_buffer) < BATCH_SIZE or rand < epsilon:
            return random.randrange(self.act_dimen)
        else:
            return np.argmax(self._predict())
    
    def learn(self, ob, act, reward, next_ob):
        if next_ob is not None:
            self._curr_stack.add_item(next_ob)
            self._prev_stack.add_item(ob)
        
        if self._curr_stack.is_full() and self._prev_stack.is_full():
            self.replay_buffer.add(self._prev_stack.get_contents(), act, reward, self._curr_stack.get_contents())
        
        self._replay()
        
        if self.act_counter % COPY_WEIGHTS_INTERVAL == 0:
            self._update_target_network()
        
    def _replay(self):
        if self.act_counter >= REPLAY_START_COUNT:
            samples = self.replay_buffer.sample(BATCH_SIZE)
            empty_state = np.zeros(self.ob_dimen)
            obs = np.asarray([s[0] for s in samples])
            next_obs = np.asarray([s[3] if s[3] is not None else empty_state for s in samples])
            
            qs = self.model.predict(obs)
            next_qs = self.target_model.predict(next_obs)
            
            # if next state of i is terminal state(next_obs[i] is None), q value is: r
            # else, q value is: r + GAMMA * max(q(s'))
            for idx, sample in enumerate(samples):
                act = sample[1]
                reward = sample[2]
                done = sample[3] is None
                if done:
                    qs[idx][act] = reward
                else:
                    qs[idx][act] = reward + GAMMA * np.max(next_qs[idx])

            self.model.fit(obs, qs, verbose=0)
            #self.model.fit(obs, qs, verbose=0, callbacks=[self.tensorboard])
            
    def _update_target_network(self):
        print("Updating Network weights.")
        self.target_model.set_weights(self.model.get_weights())
    
    def save(self, path):
        self.model.save(path)
    
    def load(self, path):
        self.model = load_model(path)

In [6]:
class Agent:
    def __init__(self, env_name = 'CartPole-v0', learner = 'dqn-v0'):
        self.env = gym.make(env_name)
        self.ob_dimen = self.env.observation_space.shape[0]
        self.act_dimen = self.env.action_space.n
        self.learner = Learner(learner, self.ob_dimen, self.act_dimen)
        self._last_frame = None
    
    def start(self, epoch_count = 1000):
        for epc in range(1, epoch_count + 1):
            total_reward = 0
            done = False
            ob = self.env.reset()
            ob = self._preprocess(ob)
            while not done:
                act = self.learner.act(ob)
                next_ob, reward, done, _ = self.env.step(act)
                if not done:
                    next_ob = self._preprocess(next_ob)
                
                self.learner.learn(ob, act, reward, None if done else next_ob)
                
                total_reward += reward
                ob = next_ob
            
            print("Epoch %s, Total Reward: %s" % (epc, total_reward))
            
    def _preprocess(self, ob):
        if self._last_frame is not None:
            img = cv2.max(ob, self._last_frame)
        else:
            img = ob
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        y, _, _ = cv2.split(img_yuv)
        o = cv2.resize(y, (84, 84))
        
        self._last_frame = ob
        return o
            
    def save(self, path):
        self.learner.save(path)
    
    def load(self, path):
        self.learner.load(path)


In [ ]:
try:
    agent = Agent(env_name='Breakout-v0')
    agent.start(100000)
finally:
    agent.save('./breakout-v0-dqn-v0.h5')

4
Updating Network weights.
Epoch 1, Total Reward: 4.0
Epoch 2, Total Reward: 1.0
Epoch 3, Total Reward: 0.0
Epoch 4, Total Reward: 0.0
Epoch 5, Total Reward: 2.0
Epoch 6, Total Reward: 0.0
Epoch 7, Total Reward: 0.0
Epoch 8, Total Reward: 2.0
Epoch 9, Total Reward: 1.0
Epoch 10, Total Reward: 3.0
Epoch 11, Total Reward: 2.0
Epoch 12, Total Reward: 1.0
Epoch 13, Total Reward: 0.0
Epoch 14, Total Reward: 1.0
Epoch 15, Total Reward: 2.0
Epoch 16, Total Reward: 2.0
Epoch 17, Total Reward: 0.0
Epoch 18, Total Reward: 2.0
Epoch 19, Total Reward: 2.0
Epoch 20, Total Reward: 0.0
Epoch 21, Total Reward: 0.0
Epoch 22, Total Reward: 1.0
Epoch 23, Total Reward: 2.0
Epoch 24, Total Reward: 1.0
Epoch 25, Total Reward: 0.0
Epoch 26, Total Reward: 0.0
Epoch 27, Total Reward: 2.0
Epoch 28, Total Reward: 0.0
Epoch 29, Total Reward: 0.0
Epoch 30, Total Reward: 0.0
Epoch 31, Total Reward: 1.0
Epoch 32, Total Reward: 1.0
Epoch 33, Total Reward: 2.0
Epoch 34, Total Reward: 0.0
Epoch 35, Total Reward: 0.0
E

Epoch 281, Total Reward: 0.0
Epoch 282, Total Reward: 1.0
Epoch 283, Total Reward: 2.0
Epoch 284, Total Reward: 1.0
Epoch 285, Total Reward: 3.0
Epoch 286, Total Reward: 3.0
Epoch 287, Total Reward: 2.0
Epoch 288, Total Reward: 1.0
Epoch 289, Total Reward: 5.0
Epoch 290, Total Reward: 1.0
Epoch 291, Total Reward: 0.0
Epoch 292, Total Reward: 6.0
Epoch 293, Total Reward: 2.0
Epoch 294, Total Reward: 0.0
Epoch 295, Total Reward: 3.0
Epoch 296, Total Reward: 1.0
Epoch 297, Total Reward: 1.0
Epoch 298, Total Reward: 2.0
Epoch 299, Total Reward: 3.0
Epoch 300, Total Reward: 0.0
Epoch 301, Total Reward: 0.0
Epoch 302, Total Reward: 2.0
Epoch 303, Total Reward: 0.0
Epoch 304, Total Reward: 2.0
Epoch 305, Total Reward: 0.0
Epoch 306, Total Reward: 3.0
Epoch 307, Total Reward: 2.0
Epoch 308, Total Reward: 2.0
Epoch 309, Total Reward: 6.0
Epoch 310, Total Reward: 2.0
Updating Network weights.
Epoch 311, Total Reward: 2.0
Epoch 312, Total Reward: 1.0
Epoch 313, Total Reward: 2.0
Epoch 314, Total 

Epoch 558, Total Reward: 0.0
Epoch 559, Total Reward: 4.0
Epoch 560, Total Reward: 1.0
Epoch 561, Total Reward: 1.0
Epoch 562, Total Reward: 2.0
Epoch 563, Total Reward: 0.0
Epoch 564, Total Reward: 0.0
Epoch 565, Total Reward: 1.0
Epoch 566, Total Reward: 4.0
Epoch 567, Total Reward: 6.0
Epoch 568, Total Reward: 0.0
Epoch 569, Total Reward: 2.0
Epoch 570, Total Reward: 0.0
Epoch 571, Total Reward: 3.0
Epoch 572, Total Reward: 0.0
Epoch 573, Total Reward: 1.0
Epoch 574, Total Reward: 1.0
Epoch 575, Total Reward: 2.0
Epoch 576, Total Reward: 1.0
Epoch 577, Total Reward: 2.0
Epoch 578, Total Reward: 3.0
Epoch 579, Total Reward: 4.0
Epoch 580, Total Reward: 2.0
Epoch 581, Total Reward: 1.0
Epoch 582, Total Reward: 1.0
Epoch 583, Total Reward: 2.0
Epoch 584, Total Reward: 0.0
Epoch 585, Total Reward: 1.0
Epoch 586, Total Reward: 2.0
Epoch 587, Total Reward: 0.0
Epoch 588, Total Reward: 2.0
Epoch 589, Total Reward: 1.0
Epoch 590, Total Reward: 0.0
Updating Network weights.
Epoch 591, Total 

Epoch 835, Total Reward: 0.0
Epoch 836, Total Reward: 0.0
Epoch 837, Total Reward: 0.0
Epoch 838, Total Reward: 1.0
Epoch 839, Total Reward: 0.0
Epoch 840, Total Reward: 0.0
Epoch 841, Total Reward: 1.0
Epoch 842, Total Reward: 2.0
Epoch 843, Total Reward: 2.0
Epoch 844, Total Reward: 2.0
Epoch 845, Total Reward: 2.0
Epoch 846, Total Reward: 2.0
Epoch 847, Total Reward: 0.0
Epoch 848, Total Reward: 0.0
Epoch 849, Total Reward: 0.0
Epoch 850, Total Reward: 2.0
Epoch 851, Total Reward: 2.0
Epoch 852, Total Reward: 4.0
Epoch 853, Total Reward: 1.0
Epoch 854, Total Reward: 3.0
Epoch 855, Total Reward: 2.0
Updating Network weights.
Epoch 856, Total Reward: 2.0
Epoch 857, Total Reward: 1.0
Epoch 858, Total Reward: 0.0
Epoch 859, Total Reward: 1.0
Epoch 860, Total Reward: 2.0
Epoch 861, Total Reward: 0.0
Epoch 862, Total Reward: 0.0
Epoch 863, Total Reward: 2.0
Epoch 864, Total Reward: 0.0
Epoch 865, Total Reward: 0.0
Epoch 866, Total Reward: 0.0
Epoch 867, Total Reward: 4.0
Epoch 868, Total 

Epoch 1108, Total Reward: 0.0
Epoch 1109, Total Reward: 0.0
Epoch 1110, Total Reward: 1.0
Epoch 1111, Total Reward: 1.0
Epoch 1112, Total Reward: 2.0
Epoch 1113, Total Reward: 0.0
Epoch 1114, Total Reward: 1.0
Epoch 1115, Total Reward: 1.0
Epoch 1116, Total Reward: 4.0
Epoch 1117, Total Reward: 2.0
Epoch 1118, Total Reward: 2.0
Updating Network weights.
Epoch 1119, Total Reward: 2.0
Epoch 1120, Total Reward: 3.0
Epoch 1121, Total Reward: 2.0
Epoch 1122, Total Reward: 2.0
Epoch 1123, Total Reward: 0.0
Epoch 1124, Total Reward: 2.0
Epoch 1125, Total Reward: 0.0
Epoch 1126, Total Reward: 0.0
Epoch 1127, Total Reward: 1.0
Epoch 1128, Total Reward: 0.0
Epoch 1129, Total Reward: 5.0
Epoch 1130, Total Reward: 3.0
Epoch 1131, Total Reward: 1.0
Epoch 1132, Total Reward: 2.0
Epoch 1133, Total Reward: 0.0
Epoch 1134, Total Reward: 0.0
Epoch 1135, Total Reward: 3.0
Epoch 1136, Total Reward: 0.0
Epoch 1137, Total Reward: 3.0
Epoch 1138, Total Reward: 1.0
Epoch 1139, Total Reward: 2.0
Epoch 1140, To

...

Epoch 13820, Total Reward: 2.0
Epoch 13821, Total Reward: 0.0
Epoch 13822, Total Reward: 1.0
Epoch 13823, Total Reward: 1.0
Epoch 13824, Total Reward: 2.0
Epoch 13825, Total Reward: 1.0
Epoch 13826, Total Reward: 0.0
Epoch 13827, Total Reward: 0.0
Epoch 13828, Total Reward: 3.0
Epoch 13829, Total Reward: 3.0
Epoch 13830, Total Reward: 1.0
Epoch 13831, Total Reward: 2.0
Epoch 13832, Total Reward: 1.0
Epoch 13833, Total Reward: 1.0
Epoch 13834, Total Reward: 2.0
Epoch 13835, Total Reward: 0.0
Epoch 13836, Total Reward: 0.0
Epoch 13837, Total Reward: 0.0
Epoch 13838, Total Reward: 1.0
Epoch 13839, Total Reward: 0.0
Epoch 13840, Total Reward: 3.0
Epoch 13841, Total Reward: 2.0
Epoch 13842, Total Reward: 2.0
Updating Network weights.
Epoch 13843, Total Reward: 0.0
Epoch 13844, Total Reward: 2.0
Epoch 13845, Total Reward: 1.0
Epoch 13846, Total Reward: 1.0
Epoch 13847, Total Reward: 1.0
Epoch 13848, Total Reward: 2.0
Epoch 13849, Total Reward: 0.0
Epoch 13850, Total Reward: 0.0
Epoch 13851, 

Epoch 14078, Total Reward: 1.0
Epoch 14079, Total Reward: 4.0
Epoch 14080, Total Reward: 3.0
Epoch 14081, Total Reward: 0.0
Epoch 14082, Total Reward: 5.0
Epoch 14083, Total Reward: 0.0
Epoch 14084, Total Reward: 0.0
Epoch 14085, Total Reward: 0.0
Epoch 14086, Total Reward: 1.0
Epoch 14087, Total Reward: 0.0
Epoch 14088, Total Reward: 0.0
Epoch 14089, Total Reward: 0.0
Updating Network weights.
Epoch 14090, Total Reward: 2.0
Epoch 14091, Total Reward: 0.0
Epoch 14092, Total Reward: 2.0
Epoch 14093, Total Reward: 1.0
Epoch 14094, Total Reward: 2.0
Epoch 14095, Total Reward: 2.0
Epoch 14096, Total Reward: 0.0
Epoch 14097, Total Reward: 1.0
Epoch 14098, Total Reward: 1.0
Epoch 14099, Total Reward: 5.0
Epoch 14100, Total Reward: 2.0
Epoch 14101, Total Reward: 1.0
Epoch 14102, Total Reward: 1.0
Epoch 14103, Total Reward: 0.0
Epoch 14104, Total Reward: 0.0
Epoch 14105, Total Reward: 1.0
Epoch 14106, Total Reward: 0.0
Epoch 14107, Total Reward: 1.0
Epoch 14108, Total Reward: 2.0
Epoch 14109, 

Epoch 14336, Total Reward: 2.0
Epoch 14337, Total Reward: 1.0
Epoch 14338, Total Reward: 0.0
Epoch 14339, Total Reward: 4.0
Epoch 14340, Total Reward: 0.0
Epoch 14341, Total Reward: 3.0
Epoch 14342, Total Reward: 0.0
Epoch 14343, Total Reward: 0.0
Epoch 14344, Total Reward: 2.0
Epoch 14345, Total Reward: 2.0
Epoch 14346, Total Reward: 1.0
Epoch 14347, Total Reward: 2.0
Updating Network weights.
Epoch 14348, Total Reward: 0.0
Epoch 14349, Total Reward: 0.0
Epoch 14350, Total Reward: 5.0
Epoch 14351, Total Reward: 4.0
Epoch 14352, Total Reward: 1.0
Epoch 14353, Total Reward: 3.0
Epoch 14354, Total Reward: 3.0
Epoch 14355, Total Reward: 2.0
Epoch 14356, Total Reward: 3.0
Epoch 14357, Total Reward: 3.0
Epoch 14358, Total Reward: 0.0
Epoch 14359, Total Reward: 3.0
Epoch 14360, Total Reward: 0.0
Epoch 14361, Total Reward: 1.0
Epoch 14362, Total Reward: 3.0
Epoch 14363, Total Reward: 4.0
Epoch 14364, Total Reward: 4.0
Epoch 14365, Total Reward: 1.0
Epoch 14366, Total Reward: 2.0
Epoch 14367, 

Epoch 14594, Total Reward: 1.0
Epoch 14595, Total Reward: 2.0
Epoch 14596, Total Reward: 0.0
Epoch 14597, Total Reward: 0.0
Epoch 14598, Total Reward: 0.0
Epoch 14599, Total Reward: 0.0
Epoch 14600, Total Reward: 1.0
Updating Network weights.
Epoch 14601, Total Reward: 2.0
Epoch 14602, Total Reward: 0.0
Epoch 14603, Total Reward: 1.0
Epoch 14604, Total Reward: 2.0
Epoch 14605, Total Reward: 4.0
Epoch 14606, Total Reward: 1.0
Epoch 14607, Total Reward: 1.0
Epoch 14608, Total Reward: 3.0
Epoch 14609, Total Reward: 5.0
Epoch 14610, Total Reward: 1.0
Epoch 14611, Total Reward: 2.0
Epoch 14612, Total Reward: 0.0
Epoch 14613, Total Reward: 3.0
Epoch 14614, Total Reward: 0.0
Epoch 14615, Total Reward: 2.0
Epoch 14616, Total Reward: 2.0
Epoch 14617, Total Reward: 3.0
Epoch 14618, Total Reward: 0.0
Epoch 14619, Total Reward: 0.0
Epoch 14620, Total Reward: 0.0
Epoch 14621, Total Reward: 0.0
Epoch 14622, Total Reward: 4.0
Epoch 14623, Total Reward: 2.0
Epoch 14624, Total Reward: 5.0
Epoch 14625, 

Epoch 14852, Total Reward: 0.0
Updating Network weights.
Epoch 14853, Total Reward: 2.0
Epoch 14854, Total Reward: 1.0
Epoch 14855, Total Reward: 4.0
Epoch 14856, Total Reward: 0.0
Epoch 14857, Total Reward: 3.0
Epoch 14858, Total Reward: 0.0
Epoch 14859, Total Reward: 2.0
Epoch 14860, Total Reward: 2.0
Epoch 14861, Total Reward: 2.0
Epoch 14862, Total Reward: 1.0
Epoch 14863, Total Reward: 1.0
Epoch 14864, Total Reward: 3.0
Epoch 14865, Total Reward: 1.0
Epoch 14866, Total Reward: 1.0
Epoch 14867, Total Reward: 3.0
Epoch 14868, Total Reward: 5.0
Epoch 14869, Total Reward: 2.0
Epoch 14870, Total Reward: 1.0
Epoch 14871, Total Reward: 0.0
Epoch 14872, Total Reward: 1.0
Epoch 14873, Total Reward: 0.0
Epoch 14874, Total Reward: 2.0
Epoch 14875, Total Reward: 2.0
Epoch 14876, Total Reward: 4.0
Epoch 14877, Total Reward: 1.0
Epoch 14878, Total Reward: 0.0
Epoch 14879, Total Reward: 2.0
Epoch 14880, Total Reward: 2.0
Epoch 14881, Total Reward: 6.0
Updating Network weights.
Epoch 14882, Total